In [ ]:
import sys
import json
from fabric import Connection
from invoke import Responder, UnexpectedExit

sys.version

In [ ]:
result = Connection('ubuntu@128.1.135.208').run('hostname', hide=False)

In [ ]:
result.stdout

In [ ]:
hosts = [
    "118.193.45.223",
    "128.1.135.208",
    "103.14.34.147",
    "23.91.97.32",
    "152.32.210.117",
    "23.91.98.101",
    "152.32.189.174",
    "36.255.220.243",
    "152.32.192.130",
    "101.36.117.63",
    "101.36.120.12",
    "101.36.122.83",
    "152.32.215.147",
    "152.32.251.236"
]

In [ ]:
# Check status

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')
    print()

In [ ]:
# All down

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./down.sh')

In [ ]:
# All up

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && git pull')
    conn.run("cd swarm-single-machine && ./up.sh")

In [ ]:
# Rename hostname

sudopass = Responder(
    pattern=r'Password:',
    response='[please change]\n')

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run(f'hostnamectl set-hostname swarm-{i:02}', pty=True, watchers=[sudopass])
    conn.run('hostnamectl')
    print()

In [ ]:
# Add self hostname to /etc/hosts
# (to solve sudo error)

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('echo $(hostname -I | cut -d\  -f1) $(hostname) | sudo tee -a /etc/hosts')
    conn.run('cat /etc/hosts')

In [ ]:
# All addresses

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./addresses.sh')

In [ ]:
# Print all eth addresses

for _, host in enumerate(hosts):
    conn = Connection(f'ubuntu@{host}')
    result = conn.run('cd swarm-single-machine && ./addresses.sh', hide=True)
    payload = json.loads(result.stdout.strip())
    print(payload['ethereum'])

In [ ]:
# Try to init all

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    try:
        conn.run('git clone https://github.com/renning22/swarm-single-machine.git && cd swarm-single-machine && ./init.sh')
    except UnexpectedExit:
        print('Skip...')

In [ ]:
# Restart unhealthy

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')
    if not result.stdout.strip():
        print('Restarting...')
        conn.run('cd swarm-single-machine && ./restart.sh')